In [1]:
%matplotlib inline
from validphys.api import API
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate as scint
from collections import defaultdict, namedtuple
import operator

from validphys.theorycovariance.construction import extract_target, compute_ratio_delta, compute_ht_parametrisation

In [2]:
fitname = "240530-01-fABMP22R-CT"
thcovmat_dict = API.fit(fit=fitname).as_input()["theorycovmatconfig"]

In [3]:
H2_coeff_list = thcovmat_dict["H2_list"]
HL_coeff_list = thcovmat_dict["HL_list"]


# dict used to produce theory predictions to construct the theory covmat as well as to produce
# theory predictions from the fit performed using the ht covmat (i.e. the predicitons that should
# be compared to data)
common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fitname,
    fits=[fitname],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
    theory={"from_": "fit"},
    theoryid={"from_": "theory"},
)

In [4]:
# collect the information (predictions + kinematics) needed for the computation of the HT covmat

# Calculate theory predictions of the input PDF
S_dict = dict(
    theorycovmatconfig={"from_": "fit"},
    pdf={"from_": "theorycovmatconfig"},
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
)
preds_ht_cov_construction = API.group_result_central_table_no_table(**(S_dict | common_dict))
preds_ht = pd.DataFrame(preds_ht_cov_construction['theory_central'])

# collect the corresponding kinemacs
process_info = API.combine_by_type_ht(**(S_dict | common_dict))
N_full_data = np.sum([i for i in process_info.sizes.values()])
kinematics_DIS = np.concatenate([v for v in [process_info.data["DIS NC"], process_info.data["DIS CC"]]]).T
# TO CHECK: IS preds[][1] THE THEORY PREDICTION?
preds_DIS = np.concatenate([v for v in [process_info.preds["DIS NC"][1], process_info.preds["DIS CC"][1]]]).T
xvals_DIS = kinematics_DIS[0]
q2vals_DIS = kinematics_DIS[1]
yvals_DIS = kinematics_DIS[2]

LHAPDF 6.5.4 loading /opt/homebrew/Caskroom/miniconda/base/envs/nnpdf/share/LHAPDF/210619-n3fit-001/210619-n3fit-001_0000.dat
210619-n3fit-001 PDF set, member #0, version 1
LHAPDF 6.5.4 loading all 101 PDFs in set 210619-n3fit-001
210619-n3fit-001, version 1; 101 PDF members


In [5]:
# Calculate theory predictions of the fit with ht covmat - this will be compared to data
preds = API.group_result_table_no_table(pdf={"from_": "fit"}, **common_dict)

LHAPDF 6.5.4 loading all 501 PDFs in set 240530-01-fABMP22R-CT
240530-01-fABMP22R-CT, version 1; 501 PDF members


In [6]:
# compute the matrix X encoding the PDF uncertainties of the predictions
preds_onlyreplicas = preds.iloc[:, 2:].to_numpy()
mean_prediction = np.mean(preds_onlyreplicas, axis=1)

X = np.zeros((preds.shape[0], preds.shape[0]))
for i in range(preds_onlyreplicas.shape[1]):
    X += np.outer(
        (preds_onlyreplicas[:, i] - mean_prediction), (preds_onlyreplicas[:, i] - mean_prediction)
    )
X *= 1 / preds_onlyreplicas.shape[1]

In [10]:
pd.options.mode.chained_assignment = None

data_by_process = API.groups_data_by_process(**(S_dict | common_dict))
PDF_thcovmat = API.pdf(**(S_dict | common_dict))

# ABMP parametrisationa
x_abmp = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]


included_proc = ["DIS NC"]
excluded_exp = {"DIS NC" : []}
included_exp = {}
for proc in included_proc:
    aux = []
    for exp in process_info.namelist[proc]:
        if exp not in excluded_exp[proc]:
            aux.append(exp)
    included_exp[proc] = aux

preds_ht.loc[['DIS NC', 'DIS CC'], 'x'] = xvals_DIS
preds_ht.loc[['DIS NC', 'DIS CC'], 'q2'] = q2vals_DIS
preds_ht.loc[['DIS NC', 'DIS CC'], 'y'] = yvals_DIS

# Initialise dataframe
for i in range(len(x_abmp)):
    preds_ht[f"p({i+1}+,0)"] = 0
    preds_ht[f"p(0,{i+1}+)"] = 0
    preds_ht[f"d({i+1}+,0)"] = 0
    preds_ht[f"d(0,{i+1}+)"] = 0

deltas = defaultdict(list)

for i_proc, proc in enumerate(process_info.namelist.keys()):
        for i_exp, exp in enumerate(process_info.namelist[proc]):
            dataset = data_by_process[i_proc].datasets[i_exp]
            kin_dict = {}

            if proc in included_proc and exp in included_exp[proc]:
                kin_dict['x']  = np.array(preds_ht.xs(exp, level=1, drop_level=False).loc[:,"x"])
                kin_dict['Q2'] = np.array(preds_ht.xs(exp, level=1, drop_level=False).loc[:,"q2"])
                kin_dict['y']  = np.array(preds_ht.xs(exp, level=1, drop_level=False).loc[:,"y"])
                kin_size =  kin_dict['x'].size
                target = extract_target(dataset)


                # Loop over the parameter
                for i in range(len(x_abmp)):
                    PC_2, PC_L = compute_ht_parametrisation(i, x_abmp, kin_dict, exp, H2_coeff_list, HL_coeff_list)
                    if target == 'proton':
                      deltas[f"p({i+1}+,0)"] += [PC_2]
                      deltas[f"p(0,{i+1}+)"] += [PC_L]
                      deltas[f"d({i+1}+,0)"] += [np.zeros(kin_size)]
                      deltas[f"d(0,{i+1}+)"] += [np.zeros(kin_size)]
                    elif target == 'deuteron':
                      deltas[f"p({i+1}+,0)"] += [np.zeros(kin_size)]
                      deltas[f"p(0,{i+1}+)"] += [np.zeros(kin_size)]
                      deltas[f"d({i+1}+,0)"] += [PC_2]
                      deltas[f"d(0,{i+1}+)"] += [PC_L]
                    elif target == 'ratio':
                      deltas[f"p({i+1}+,0)"] += [compute_ratio_delta(dataset, PDF_thcovmat, "p", PC_2) - compute_ratio_delta(dataset, PDF_thcovmat)]
                      deltas[f"p(0,{i+1}+)"] += [compute_ratio_delta(dataset, PDF_thcovmat, "p", PC_L) - compute_ratio_delta(dataset, PDF_thcovmat)]
                      deltas[f"d({i+1}+,0)"] += [compute_ratio_delta(dataset, PDF_thcovmat, "d", PC_2) - compute_ratio_delta(dataset, PDF_thcovmat)]
                      deltas[f"d(0,{i+1}+)"] += [compute_ratio_delta(dataset, PDF_thcovmat, "d", PC_L) - compute_ratio_delta(dataset, PDF_thcovmat)]
                    else:
                        raise ValueError("Could not detect target.")
            else:
                for i in range(len(x_abmp)):
                    deltas[f"p({i+1}+,0)"] += [np.zeros(preds_ht.xs(exp, level=1, drop_level=False).shape[0])]
                    deltas[f"p(0,{i+1}+)"] += [np.zeros(preds_ht.xs(exp, level=1, drop_level=False).shape[0])]
                    deltas[f"d({i+1}+,0)"] += [np.zeros(preds_ht.xs(exp, level=1, drop_level=False).shape[0])]
                    deltas[f"d(0,{i+1}+)"] += [np.zeros(preds_ht.xs(exp, level=1, drop_level=False).shape[0])]

delta_pred = []
for i in range(len(x_abmp)):
    temp_1 = np.array([])
    temp_2 = np.array([])
    temp_3 = np.array([])
    temp_4 = np.array([])
    for vec in zip(deltas[f"p({i+1}+,0)"], deltas[f"p(0,{i+1}+)"], deltas[f"d({i+1}+,0)"], deltas[f"d(0,{i+1}+)"]):
        temp_1 = np.concatenate((temp_1, vec[0]))
        temp_2 = np.concatenate((temp_2, vec[1]))
        temp_3 = np.concatenate((temp_3, vec[2]))
        temp_4 = np.concatenate((temp_4, vec[3]))
    
    preds_ht[f"p({i+1}+,0)"] = temp_1
    preds_ht[f"p(0,{i+1}+)"] = temp_2
    preds_ht[f"d({i+1}+,0)"] = temp_3
    preds_ht[f"d(0,{i+1}+)"] = temp_4
    delta_pred.append(preds_ht[f"p({i+1}+,0)"])
    delta_pred.append(preds_ht[f"p(0,{i+1}+)"])
    delta_pred.append(preds_ht[f"d({i+1}+,0)"])
    delta_pred.append(preds_ht[f"d(0,{i+1}+)"])
    


In [11]:
# Theory covariance matrix
S = np.zeros((delta_pred[0].size, delta_pred[0].size))
for delta in delta_pred:
    S += np.outer(delta, delta)

S = pd.DataFrame(S, index=delta_pred[0].index, columns=delta_pred[0].index)

# Experimental covariance matrix
C = API.groups_covmat_no_table(**common_dict)

# Ensure that S anc C are ordered in the same way (in practice they already are)
S = S.reindex(C.index).T.reindex(C.index)

In [ ]:
# Load the central value of the pseudodata
# this is needed to compute the distance between prediction and data
pseudodata = API.read_pdf_pseudodata(**common_dict)
dat_central = np.mean(
    [i.pseudodata.reindex(preds.index.to_list()).to_numpy().flatten() for i in pseudodata],
    axis=0,
)

In [ ]:
# Compute delta_T_tilde (Eq. 3.37) and P_tilde (Eq. 3.38) of arXiv:2105.05114

# The factors 1/sqrt(2) are to normalize for the fact that beta provides information about
# theoretical uncertainties along two directions
# CHECK THIS PART
# b_tilde SHOULD BE INDEPENDENT OF THE PRIOR THAT WE USE TO MODEL HT CORRECTIONS.

central_ht_coeffs = np.zeros(len(H2_coeff_list) + len(HL_coeff_list)) 

# Construct beta tilde
H_single_list = np.concatenate((H2_coeff_list, HL_coeff_list))
beta_tilde = []
for i, par in enumerate(H_single_list):
  aux = np.zeros(H_single_list.size)
  aux[i] = par
  beta_tilde.append(aux)

S_tilde = np.zeros((len(beta_tilde[0]), len(beta_tilde[0])))
for tilde in beta_tilde:
    S_tilde += np.outer(tilde,tilde)

beta = delta_pred
S_hat = np.zeros((len(beta_tilde[0]),delta_pred[0].size))
for b in zip(beta_tilde, beta):
    S_hat += np.outer(b[0], b[1])

invcov = np.linalg.inv(C + S)

delta_T_tilde = -S_hat @ invcov @ (mean_prediction - dat_central)
# where are the X_tilde and X_hat terms in P_tilde?
# Maybe not present because we don't have correlations between theory parameters
P_tilde = S_hat @ invcov @ X @ invcov @ S_hat.T + (S_tilde - S_hat @ invcov @ S_hat.T)
preds = central_ht_coeffs + delta_T_tilde
uncs = np.sqrt(P_tilde)

In [ ]:
# check if the stored covmat is equal to S we recomputed above
fitpath = API.fit(fit=fitname).path
try:
    stored_covmat = pd.read_csv(
        fitpath / "tables/datacuts_theory_theorycovmatconfig_user_covmat.csv",
        sep="\t",
        encoding="utf-8",
        index_col=2,
        header=3,
        skip_blank_lines=False,
    )
except FileNotFoundError:
    stored_covmat = pd.read_csv(
        fitpath / "tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
        index_col=[0, 1, 2],
        header=[0, 1, 2],
        sep="\t|,",
        engine="python",
    ).fillna(0)
    storedcovmat_index = pd.MultiIndex.from_tuples(
        [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_covmat.index],
        names=["group", "dataset", "id"],
    )
    stored_covmat = pd.DataFrame(
        stored_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
    )
    stored_covmat = stored_covmat.reindex(S.index).T.reindex(S.index)


In [ ]:
# print the final result
if np.allclose(S, stored_covmat):
    print(
        f"Reversed 5pt\n"
        f"-----------------------------\n"
        )
    for i, pred in enumerate(preds):
        tpye = "2" if i < len(H2_coeff_list) else "L"
        n = i%7
        print(
            f"H_{tpye} node {n+1} = {preds[i]:.5f} ± {np.sqrt(P_tilde[i,i]):.5f} \n"
        )
        if i == len(H2_coeff_list)-1:
            print("-----------------------------\n")
else:
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

# Plots

In [ ]:
y2_central = [preds[i] for i in range(len(H2_coeff_list))]
y2_sigma = [np.sqrt(P_tilde[i,i]) for i in range(len(H2_coeff_list))]
y2_plus = [x1 + x2 for x1,x2 in zip(y2_central, y2_sigma)]
y2_minus = [x1 - x2 for x1,x2 in zip(y2_central, y2_sigma)]

yL_central = [preds[i] for i in range(len(H2_coeff_list), len(HL_coeff_list) + len(H2_coeff_list))]
yL_sigma = [np.sqrt(P_tilde[i,i]) for i in range(len(H2_coeff_list), len(HL_coeff_list) + len(H2_coeff_list))]
yL_plus = [x1 + x2 for x1,x2 in zip(yL_central, yL_sigma)]
yL_minus = [x1 - x2 for x1,x2 in zip(yL_central, yL_sigma)]


In [ ]:
H2 = sp.interpolate.CubicSpline(x_abmp, y2_central)
H2_plus = sp.interpolate.CubicSpline(x_abmp, y2_plus)
H2_minus = sp.interpolate.CubicSpline(x_abmp, y2_minus)
H2_color = "sandybrown"
H2_label = "H2"


HL = sp.interpolate.CubicSpline(x_abmp, yL_central)
HL_plus = sp.interpolate.CubicSpline(x_abmp, yL_plus)
HL_minus = sp.interpolate.CubicSpline(x_abmp, yL_minus)
HL_color = "green"
HL_label = "HL"

In [4]:
def plot_wrapper(H, H_p, H_m, y, label, color, ylabel):
  xv = np.logspace(-5, -0.0001, 100)
  legends = []
  legend_name = [label, "knots"]
  fig, ax = plt.subplots(figsize=(12.5, 8))
  knots = ax.plot(x_abmp, y, 'o', label='data')
  pl = ax.plot(xv, H(xv), ls = "-", lw = 1, color=color)
  pl_lg= ax.fill(np.NaN, np.NaN, color = color, alpha = 0.3) # Necessary for fancy legend
  legends.append((pl[0], pl_lg[0]))
  legends.append(knots[0])
  ax.fill_between(xv, H_p(xv), H_m(xv), color = color, alpha = 0.3)
  ax.set_xscale("log")
  ax.set_xlabel(f'$x$')
  ax.set_ylabel(ylabel)
  
  
  fig.legend(legends, legend_name, loc=[0.1,0.15], fontsize=15)
  plt.show()

In [ ]:
plot_wrapper(H2, H2_plus, H2_minus, y2_central, label=r"$H_2 \pm \sigma$", color=H2_color, ylabel=r"$H_2$")

In [ ]:
plot_wrapper(HL, HL_plus, HL_minus, yL_central, label=r"$H_L \pm \sigma$", color=HL_color, ylabel=r"$H_L$")

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 7))
H2_dict = {
  "func" : H2,
  "func_plus_std": H2_plus,
  "func_minus_std": H2_minus,
  "knots": y2_central,
  "label": r"$H_2 \pm \sigma$",
  "ylabel": r"$H_2$",
  "color": H2_color
}

HL_dict = {
  "func" : HL,
  "func_plus_std": HL_plus,
  "func_minus_std": HL_minus,
  "knots": yL_central,
  "label": r"$H_L \pm \sigma$",
  "ylabel": r"$H_L$",
  "color": HL_color
}
dicts = [H2_dict, HL_dict]

def plot_wrapper(H, H_p, H_m, y, label, color, ylabel, ax):
  xv = np.logspace(-5, -0.0001, 100)
  legends = []
  legend_name = [label, "knots"]
  knots = ax.plot(x_abmp, y, 'o', label='data')
  pl = ax.plot(xv, H(xv), ls = "-", lw = 1, color=color)
  pl_lg= ax.fill(np.NaN, np.NaN, color = color, alpha = 0.3) # Necessary for fancy legend
  legends.append((pl[0], pl_lg[0]))
  legends.append(knots[0])
  ax.fill_between(xv, H_p(xv), H_m(xv), color = color, alpha = 0.3)
  ax.set_xscale("log")
  ax.set_xlabel(f'$x$')
  ax.set_ylabel(ylabel)
  ax.legend(legends, legend_name, loc=[0.1,0.15], fontsize=15)

for i, HTdict in enumerate(dicts):
  plot_wrapper(H=HTdict["func"],
               H_p=HTdict["func_plus_std"],
               H_m=HTdict["func_minus_std"],
               y=HTdict["knots"],
               label=HTdict["label"],
               color=HTdict["color"],
               ylabel=HTdict["ylabel"],
               ax=axs[i])

axs[0].text(5.e-5, 0.06, fitname, fontsize=20)
fig.savefig()
plt.show()